In [1]:
import itertools as it
import os
import pandas as pd
import numpy as np
import random
import vitaldb
from pyvital.pyvital import arr
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import time, datetime
import neurokit2 as nk

SRATE = 300
LEN_INPUT = 60
OVERLAP = 10
LEN_PER_PRE = 60
LEN_PER_POST = 60


print(datetime.datetime.now())

2022-11-17 12:31:37.833320


In [2]:
from keras import losses, metrics
from keras.models import Sequential
from keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization, Dropout, Activation, Input
from keras.layers import GlobalAveragePooling1D, Flatten, SeparableConv1D, concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score, precision_recall_curve
import tensorflow as tf
import os, sys, pickle
import pandas as pd
import random, datetime, time
from sklearn.model_selection import KFold


#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "3"

print(datetime.datetime.now())

2022-11-17 12:31:39.826032


In [3]:
print('loading train...', flush=True, end='')

# x를 loading해서 (batch_size, step, channel)
LEN_INPUT = 60
SRATE = 300
ECG_FILT = 'bandpass'
PPG_FILT = 'lowess'
CORR_THRES = 0.9
input_path = f"../DL_model/dataset/PD_SRATE{SRATE}-PPG-{PPG_FILT}_ECG-{ECG_FILT}_THRES{CORR_THRES}/"


x_train = np.load(input_path+'x_train.npz', allow_pickle=True)['arr_0']
x_test = np.load(input_path+'x_test.npz', allow_pickle=True)['arr_0']
rftn_train = np.load(input_path+'rftn_train.npz')['arr_0']
rftn_test = np.load(input_path+'rftn_test.npz')['arr_0']
y_train = np.load(input_path+'y_train.npz')['arr_0']
y_test = np.load(input_path+'y_test.npz')['arr_0']

y_train_bin = y_train > 0.25
y_test_bin = y_test > 0.25

gender_train = np.load(input_path+'gender_train.npz', allow_pickle=True)['arr_0']
gender_test = np.load(input_path+'gender_test.npz', allow_pickle=True)['arr_0']
age_train = np.load(input_path+'age_train.npz', allow_pickle=True)['arr_0']
age_test = np.load(input_path+'age_test.npz', allow_pickle=True)['arr_0']

rftn_train = np.reshape(rftn_train, [-1,LEN_INPUT*SRATE,1])
input_train = np.concatenate((x_train, rftn_train), axis=2)
rftn_test = np.reshape(rftn_test, [-1,LEN_INPUT*SRATE,1])
input_test = np.concatenate((x_test, rftn_test), axis=2)


print('done', flush=True)

print('input_train shape:', input_train.shape)
print('input_test.shape:', input_test.shape)

print(f'y_train : mean {y_train.mean():.2f}, max {y_train.max():.2f}, min {y_train.min():.2f}')
print(f'y_test : mean {y_test.mean():.2f}, max {y_test.max():.2f}, min {y_test.min():.2f}')

print(datetime.datetime.now())

loading train...done
input_train shape: (1056, 18000, 3)
input_test.shape: (117, 18000, 3)
y_train : mean 0.21, max 0.89, min -1.00
y_test : mean 0.21, max 0.78, min -0.22
2022-11-17 12:31:43.418029


In [8]:
train_w_samp = np.ones(shape=(len(y_train),))
#val_w_samp = np.ones(shape=(len(y_val),))
test_w_samp = np.ones(shape=(len(y_test),))


# folder
nfold = 5  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 500
rootdir = f"../DL_model/randomSearch/HR_PD/CNN_HR0.25_Bin_SRATE{SRATE}-PPG-{PPG_FILT}_ECG-{ECG_FILT}_THRES{CORR_THRES}_{nfold}fold_test{ntest}_w_samp0(train)"

if not os.path.exists(rootdir):
    os.mkdir(rootdir)

# 모델에 대한 정보 txt로 저장
f = open(f'{rootdir}/README.txt', 'w')
f.write(f'model: 1D CNN 4 layers, binary classification')
f.write(f'input: ECG,PPG,RFTN of pre-intubation 120~60s, output: del HR 0.25 (post-intubation 0-60s)')
f.close()
    

# test_settings
layer_settings, test_settings = [], []


# hyperparamters
#num_nodes = [64, 64, 64] #, 64, 64, 64]
#kernel_size = 10
pool_size = 2

#dense_node = 32
#dropout_rate = 0.2
learning_rate = 0.001

# hyperparamters pool
filt_opts = [32, 64, 128] # num of filters(kernel)
stride_opts = [1,1,1,1,1,2,2,2,2]
kernel_opts = range(3,9,2) # kernel size
dropout_opts  = [0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [0, 8, 16, 32, 64]
globalpool_opts = ['max','ave']
BATCH_SIZE = [256, 512, 1024]

print('start making test settings...', end='', flush=True)
# test settings
nfilt, kernels, strides = [], [], []
for i in range(5):
    nfilt.append(0)
    kernels.append(0)
    strides.append(0)

for nfilter in filt_opts:
    for kernel in kernel_opts:
        for stride in stride_opts:
            layer_settings.append([nfilter, kernel, stride])                
    
for dense_node in dense_opts:
    for dropout_cnn in dropout_opts:
        for dropout_fc in dropout_opts:
            for globalpool_opt in globalpool_opts:
                for batch_size in BATCH_SIZE:
                    for conv_double in [True, False]:
                        for learning_rate in [0.001, 0.002, 0.0005]:
                            test_settings.append([dense_node, dropout_cnn, dropout_fc, globalpool_opt, batch_size, conv_double, learning_rate])                                   

                        
print('done')
print(datetime.datetime.now())

start making test settings...done
2022-11-17 12:36:18.635907


In [10]:
from keras import metrics
from scipy import interp
from sklearn.metrics import roc_curve,auc

# random search for hyperparameter
ntrial = ntest
train_errs, val_errs = [] ,[]
test_acc, test_roc, test_prc = [], [], []
#test_rmse, test_mae, test_auc = [], [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    
    # total conv layers of the model
    n_conv = random.choice([2,3,4]) 
    # test settings
    for i in range(n_conv):
        nfilt[i], kernels[i], strides[i] = random.choice(layer_settings)
    dense_node, dropout_cnn, dropout_fc, globalpool_opt, batch_size, conv_double, learning_rate = random.choice(test_settings)
    

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = f'batch{batch_size},'
    for i in range(n_conv):
        odir_f += f'layer{i+1}:c{nfilt[i]}filt{kernels[i]}str{strides[i]},'
    odir_f += f'globalpool={globalpool_opt},dropout{dropout_cnn},dnodes{dense_node},dropout{dropout_fc}'#,lr{learning_rate}'
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = f"{odir}/weights.hdf5"    

    
    # build a model
    act='relu'

    inp = Input(shape=(x_train.shape[1],3))
    out = inp

    for i in range(n_conv):
        out = Conv1D(filters=nfilt[i], kernel_size=kernels[i], strides=strides[i], padding='same')(out)
        out = BatchNormalization()(out)
        out = MaxPooling1D(pool_size)(out)



    # global이냐 flatten이냐는 따로 모델 나눠야 할듯
    if globalpool_opt == 'max':
        out = GlobalMaxPooling1D()(out)
    elif globalpool_opt == 'ave':
        out = GlobalAveragePooling1D()(out)


    if dense_node != 0:
        out = Dropout(dropout_cnn)(out)
        out = Dense(dense_node, activation='tanh')(out)
    out = Dropout(dropout_fc)(out)
    out = Dense(1, activation='sigmoid')(out)


    model = Model(inputs=[inp], outputs=[out])

    # model 학습 설정

    kfold = KFold()
    cv_scores = []
    tprs, aucs, prs = [], [], []
    pr_aucs, roc_aucs = [], []
    mean_fpr = np.linspace(0,1,100)
    mean_recall = np.linspace(0,1,100)
    idx = 1
    for cv_train_mask, cv_test_mask in kfold.split(input_train, y_train_bin):
        try:
            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc", tf.keras.metrics.AUC()])
            hist = model.fit(input_train[cv_train_mask], y_train_bin[cv_train_mask], sample_weight=train_w_samp[cv_train_mask], validation_split=0.2, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])
            #_, cv_score = model.evaluate(input_train[cv_test_mask], y_train_bin[cv_test_mask], batch_size=batch_size, verbose=0)

            # ROC analysis
            fpr, tpr, t = roc_curve(y_train_bin[cv_test_mask], model.predict(input_train[cv_test_mask]))
            tprs.append(interp(mean_fpr, fpr, tpr))
            roc_auc = auc(fpr, tpr)
            roc_aucs.append(roc_auc)
            # PR analysis
            prec, recall, th = precision_recall_curve(y_train_bin[cv_test_mask], model.predict(input_train[cv_test_mask]))
            prs.append(interp(mean_recall, prec, recall))
            pr_auc = auc(recall, prec)
            pr_aucs.append(pr_auc)

            print(f' ###{idx} fold : val auc {roc_auc:.3f}, val prc {pr_auc:.3f}###')
            #cv_scores.append(cv_score)

        except Exception as e:
            print(e)
            os.rmdir(odir)
            #os.rename(odir,rootdir+'/error_{}'.format(odir_f))
            itrial -= 1
            test_roc.append(0)
            test_acc.append(0)
            test_prc.append(0)
            train_errs.append(-1)
            val_errs.append(-1)
            continue

    # mean ROC
    mean_tpr = np.mean(tprs, axis = 0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_roc = np.std(roc_aucs)
    # mean PRC
    mean_precision = np.mean(prs, axis = 0)
    mean_prc = auc(mean_recall, mean_precision)
    std_prc = np.std(pr_aucs)
    
                     
    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())



    # rename
    os.rename(odir, rootdir+f'/roc{mean_auc:.3f}+-{std_roc:.3f}_prc{mean_prc:.3f}+-{std_prc:.3f}_{odir_f}')



    tf.keras.backend.clear_session()




random search 0/500
Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 0.7283 - acc: 0.5570 - auc_2: 0.5393
Epoch 00001: val_loss improved from inf to 1.86479, saving model to ../DL_model/randomSearch/HR_PD/CNN_HR0.25_Bin_SRATE300-PPG-lowess_ECG-bandpass_THRES0.9_5fold_test500_w_samp0(train)/batch1024,layer1:c64filt7str2,layer2:c128filt5str2,layer3:c128filt7str2,globalpool=ave,dropout0.5,dnodes64,dropout0.3/weights.hdf5
1/1 [==============================] - 3s 3s/step - loss: 0.7283 - acc: 0.5570 - auc_2: 0.5393 - val_loss: 1.8648 - val_acc: 0.4438 - val_auc_2: 0.5660
Epoch 2/100
1/1 [==============================] - ETA: 0s - loss: 1.1462 - acc: 0.4711 - auc_2: 0.4600
Epoch 00002: val_loss improved from 1.86479 to 0.71429, saving model to ../DL_model/randomSearch/HR_PD/CNN_HR0.25_Bin_SRATE300-PPG-lowess_ECG-bandpass_THRES0.9_5fold_test500_w_samp0(train)/batch1024,layer1:c64filt7str2,layer2:c128filt5str2,layer3:c128filt7str2,globalpool=ave,dropout0.5,dnodes64,dropout0

/tmp/ipykernel_993142/283457204.py:90: DeprecationWarning: scipy.interp is deprecated and will be removed in SciPy 2.0.0, use numpy.interp instead
  tprs.append(interp(mean_fpr, fpr, tpr))
/tmp/ipykernel_993142/283457204.py:95: DeprecationWarning: scipy.interp is deprecated and will be removed in SciPy 2.0.0, use numpy.interp instead
  prs.append(interp(mean_recall, prec, recall))


 ###1 fold : val auc 0.511, val prc 0.326###
Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 0.7787 - acc: 0.5429 - auc_3: 0.5153
Epoch 00001: val_loss improved from inf to 0.79761, saving model to ../DL_model/randomSearch/HR_PD/CNN_HR0.25_Bin_SRATE300-PPG-lowess_ECG-bandpass_THRES0.9_5fold_test500_w_samp0(train)/batch1024,layer1:c64filt7str2,layer2:c128filt5str2,layer3:c128filt7str2,globalpool=ave,dropout0.5,dnodes64,dropout0.3/weights.hdf5
1/1 [==============================] - 3s 3s/step - loss: 0.7787 - acc: 0.5429 - auc_3: 0.5153 - val_loss: 0.7976 - val_acc: 0.4497 - val_auc_3: 0.5661
Epoch 2/100
1/1 [==============================] - ETA: 0s - loss: 0.7309 - acc: 0.5385 - auc_3: 0.5184
Epoch 00002: val_loss improved from 0.79761 to 0.77211, saving model to ../DL_model/randomSearch/HR_PD/CNN_HR0.25_Bin_SRATE300-PPG-lowess_ECG-bandpass_THRES0.9_5fold_test500_w_samp0(train)/batch1024,layer1:c64filt7str2,layer2:c128filt5str2,layer3:c128filt7str2,globalpool=ave,dro

/tmp/ipykernel_993142/283457204.py:90: DeprecationWarning: scipy.interp is deprecated and will be removed in SciPy 2.0.0, use numpy.interp instead
  tprs.append(interp(mean_fpr, fpr, tpr))
/tmp/ipykernel_993142/283457204.py:95: DeprecationWarning: scipy.interp is deprecated and will be removed in SciPy 2.0.0, use numpy.interp instead
  prs.append(interp(mean_recall, prec, recall))


 ###1 fold : val auc 0.543, val prc 0.391###
Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 0.6843 - acc: 0.5680 - auc_4: 0.5504
Epoch 00001: val_loss improved from inf to 0.91218, saving model to ../DL_model/randomSearch/HR_PD/CNN_HR0.25_Bin_SRATE300-PPG-lowess_ECG-bandpass_THRES0.9_5fold_test500_w_samp0(train)/batch1024,layer1:c64filt7str2,layer2:c128filt5str2,layer3:c128filt7str2,globalpool=ave,dropout0.5,dnodes64,dropout0.3/weights.hdf5
1/1 [==============================] - 3s 3s/step - loss: 0.6843 - acc: 0.5680 - auc_4: 0.5504 - val_loss: 0.9122 - val_acc: 0.5680 - val_auc_4: 0.4862
Epoch 2/100
1/1 [==============================] - ETA: 0s - loss: 0.6937 - acc: 0.6036 - auc_4: 0.5789
Epoch 00002: val_loss improved from 0.91218 to 0.71264, saving model to ../DL_model/randomSearch/HR_PD/CNN_HR0.25_Bin_SRATE300-PPG-lowess_ECG-bandpass_THRES0.9_5fold_test500_w_samp0(train)/batch1024,layer1:c64filt7str2,layer2:c128filt5str2,layer3:c128filt7str2,globalpool=ave,dro

...

2022-11-17 15:42:02.768771: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.90GiB (rounded to 3110400000)requested by op gradient_tape/model/conv1d_1/Conv1D/Conv2DBackpropInput
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-11-17 15:42:02.768862: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-11-17 15:42:02.768894: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 238, Chunks in use: 238. 59.5KiB allocated for chunks. 59.5KiB in use in bin. 12.4KiB client-requested in use in bin.
2022-11-17 15:42:02.768915: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 114, Chunks in use: 111. 65.2KiB allocated for chunks. 63.8KiB in use in bin. 56.9KiB client-re

 OOM when allocating tensor with shape[675,128,1,9000] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/model/conv1d_1/Conv1D/Conv2DBackpropInput
 (defined at /home/painstudy/anaconda3/envs/keras/lib/python3.8/site-packages/tensorflow/python/util/tf_stack.py:193)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_1449221]

Errors may have originated from an input operation.
Input Source operations connected to node gradient_tape/model/conv1d_1/Conv1D/Conv2DBackpropInput:
In[0] gradient_tape/model/conv1d_1/Conv1D/ShapeN:	
In[1] model/conv1d_1/Conv1D/ExpandDims_1:	
In[2] gradient_tape/model/conv1d_1/Conv1D/Reshape:

Operation defined at: (most recent call last)
>>>   File "/home/painstudy/anaconda3/envs/keras/lib/python3.8/runpy.py", line 194, in

2022-11-17 15:42:17.282507: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.90GiB (rounded to 3115008000)requested by op gradient_tape/model/conv1d_1/Conv1D/Conv2DBackpropInput
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-11-17 15:42:17.282616: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-11-17 15:42:17.282646: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 218, Chunks in use: 211. 54.5KiB allocated for chunks. 52.8KiB in use in bin. 8.8KiB client-requested in use in bin.
2022-11-17 15:42:17.282666: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 83, Chunks in use: 81. 48.2KiB allocated for chunks. 47.0KiB in use in bin. 40.7KiB client-reque

 OOM when allocating tensor with shape[676,128,1,9000] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/model/conv1d_1/Conv1D/Conv2DBackpropInput
 (defined at /home/painstudy/anaconda3/envs/keras/lib/python3.8/site-packages/tensorflow/python/util/tf_stack.py:193)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_1454014]

Errors may have originated from an input operation.
Input Source operations connected to node gradient_tape/model/conv1d_1/Conv1D/Conv2DBackpropInput:
In[0] gradient_tape/model/conv1d_1/Conv1D/ShapeN:	
In[1] model/conv1d_1/Conv1D/ExpandDims_1:	
In[2] gradient_tape/model/conv1d_1/Conv1D/Reshape:

Operation defined at: (most recent call last)
>>>   File "/home/painstudy/anaconda3/envs/keras/lib/python3.8/runpy.py", line 194, in

FileNotFoundError: [Errno 2] No such file or directory: '../DL_model/randomSearch/HR_PD/CNN_HR0.25_Bin_SRATE300-PPG-lowess_ECG-bandpass_THRES0.9_5fold_test500_w_samp0(train)/batch1024,layer1:c128filt3str1,layer2:c64filt7str2,layer3:c128filt5str2,globalpool=max,dropout0.5,dnodes16,dropout0.1'

In [ ]:
    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(input_test).flatten()

    
    # acc 계산
    acc = metrics.Accuracy()
    acc.update_state(y_pred>=0.5, y_test_bin, sample_weight=test_w_samp)
    acc_val = acc.result().numpy()
    test_acc.append(acc_val)
    
    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test_bin, y_pred, sample_weight=test_w_samp)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_roc.append(roc_auc)

    # auprc 
    precision, recall, _ = precision_recall_curve(y_test_bin, y_pred, sample_weight=test_w_samp)
    prc_auc = auc(recall, precision)
    test_prc.append(prc_auc)

    
    # rename
    os.rename(odir, rootdir+'/roc{:.4f}_prc{:.4f}_{}_acc{:.2f}'.format(roc_auc, prc_auc, odir_f, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_roc.index(max(test_roc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_roc[max_idx], random_settings[max_idx]))
